In [1]:
%%html
<style>table {float:left}</style>

In [2]:
import sys
print('python version:', sys.version_info)
import sqlalchemy
print('sqlalchemy version:',sqlalchemy.__version__)

python version: sys.version_info(major=3, minor=6, micro=10, releaselevel='final', serial=0)
sqlalchemy version: 1.3.13


 # Pythonとデータベース
Pythonから利用できるデータベースはMySQL,PostgreSQL, SQLite,Oracleなど多岐にわたります。
PythonのデータベースAPI仕様は、PEP 249※1で定義されており、それぞれのデータベースに対応して実装※2が存在します。

データベースAPIを使用すればデータベースプログラミンが可能ですが、SQLを直接記述する必要があります。
SQLを直接記述するプログラムは規模が大きくなると複雑になる傾向があります。そのため最近のデータベース開発では
オブジェクトとデータベースのレコードを関連付けるしくみであるO/Rマッパーを利用してデータベース開発を行うケースが増えています。

※1 https://www.python.org/dev/peps/pep-0249/#exceptions <br>
※2 https://docs.python.org/ja/3/library/sqlite3.html

# SQLAlchemy
SQLAlchemyは、データベースやSQLに関連する機能を提供するライブラリです。テーブル関連の処理やSQLを扱うための便利なモジュールを提供しています。
O/Rマッパーは提供される機能の１つであり、O/Rマッパーを使用せずに開発することも可能です。SQLAlchemyではDataMapperパターンを使用してテーブルとオブジェクトのマッピングを行い、開発を進めます。

SQLAlchmeyは非常に高機能であり、SQLite,MySQL,PostgerSQLなどのオープンソースのデータベースや、OracleやMicrosoft SQL Serverなどの商用データベースもサポートしています。

# データベースへの接続
データベースの接続は、create_engine()関数を使用します。第１引数は接続するデータベースを指定します。ここでは、メモリーベースのSQLite3を指定しています。
第2引数のechoはログ出力の有効化フラグです。Trueを指定するとコンソールにSQLAlchemyが発行したSQLなどが出力されます。

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)
engine

Engine(sqlite:///:memory:)

# テーブルの作成
SQLAlchemyを使用してテーブルを作成します。テーブルは次の表のように書籍の情報を格納するbooksテーブルを作成します。SQLAlchemyでは、テーブルを表すオブジェクトとマッピングされるオブジェクトを定義し、それらを関連付けます。
定義と関連付けはdeclarativeを使って同時に行っています。
#### booksテーブルの定義
|<div align='left'>名前</div>|<div align='left'>カラム名</div>|<div align='left'>データタイプ</div>
|-----|-----|-----
|<div align='left'>ID</div>|<div align='left'>id</div>|<div align='left'>INTEGER</div>
|<div align='left'>タイトル</div>|<div align='left'>title</div>|<div align='left'>VARCHAR(255)</div>
|<div align='left'>価格</div>|<div align='left'>price</div>|<div align='left'>INTEGER</div>
|<div align='left'>メモ</div>|<div align='left'>memo</div>|<div align='left'>TEXT</div>

### Bookクラス（オブジェクト）定義

Bookクラスの定義は次のように行います。
1. declarativeをインポートします。
2. declarative.declarative_base()関数でBaseクラスを生成します。
3. フィールド定義に使用する Column, Integer, Unicode, UnicodeTextをインポートします。
4. Baseクラスを継承したBookクラスを生成します。Bookクラスでは、データベースのカラムに対応するフィールドの定義を行います。
5. \_\_tablename\_\_ = 'books' でテーブル名を指定します。
6. Columnクラスを使ってフィールドを定義します。

Columnクラスのコンストラクタの第１引数には、フィールドの型、第２引数以降にはオプションを指定します。
primary_key=Trueと指定すると、そのカラムはプライマリーキーとなります。nullable=Falseと指定すると、nullが許可さなくなります。
また、テーブルにデータが保存されるときのでフィルと値はdefault='<デフォルト値>'で指定します。

declarativeを使ってクラス宣言したので、booksテーブルとマッピングされたBookオブジェクトがプログラム内で使用できるようになります。


In [4]:
from sqlalchemy.ext import declarative
Base = declarative.declarative_base()
from sqlalchemy import Column, Integer, Unicode, UnicodeText
class Book(Base):
    __tablename__ = 'books'
    id = Column(Integer, primary_key=True)
    title = Column(Unicode(100), nullable=False)
    price = Column(Integer, nullable=False)
    memo = Column(UnicodeText)
    def __repr__(self):
        return "<Book('%s', '%s', '%s')>" % (self.title, self.price, self.memo)

### テーブル作成
テーブルは、Base.metadatのcreate_all()メソッドにcreate_engine() で作成したengineを渡して作成します。
Metadateはテーブルなどの情報を登録するためのオブジェクトです。create_all()メソッドを呼び出すとデータベース上にテーブルが作成されます。

In [5]:
Base.metadata.create_all(engine)

2020-03-24 14:19:20,171 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-03-24 14:19:20,173 INFO sqlalchemy.engine.base.Engine ()
2020-03-24 14:19:20,176 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-03-24 14:19:20,176 INFO sqlalchemy.engine.base.Engine ()
2020-03-24 14:19:20,177 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("books")
2020-03-24 14:19:20,178 INFO sqlalchemy.engine.base.Engine ()
2020-03-24 14:19:20,179 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("books")
2020-03-24 14:19:20,179 INFO sqlalchemy.engine.base.Engine ()
2020-03-24 14:19:20,180 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE books (
	id INTEGER NOT NULL, 
	title VARCHAR(100) NOT NULL, 
	price INTEGER NOT NULL, 
	memo TEXT, 
	PRIMARY KEY (id)
)


2020-03-24 14:19:20,181 INFO sqlalchemy.engine.base.Engine ()
2020-03-24 14:19:20,183 INFO sqlalchemy.engine.base.Engine COMMIT


# テーブルの操作
テーブルとオブジェクトのマッピングが完了すると、データベースにアクセスして基本的なCRUD操作ができるようになります。<br>
CRUDとは、__C__reate(登録)、__R__ead（読み取り）、__U__pdate(更新）、__D__elete（削除）の略称です。
### セッションの作成
SQLAlchemyではセッションを通してオブジェクトの取得や更新を行います。まずは次に示すようにSessionインスタンスを作成します。
1. sqlchemey.ormのsessionmakerをインポートします。
2. sessionmakerでengineを関連付け(bind)してSessionクラスを作成します。
3. Sessionクラスのコンストラクターを呼び出してインスタンスsessionを生成します。

In [6]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

### トランザクション
SQLAlchemyでは、オブジェクトの登録、更新、削除を行った後、session.commit()メソッド呼び出してオブジェクトの更新をデータベースに反映します。
また何らかのエラーが発生してトランザクションをロールバックしたい場合には、session.rollback()メソッドを呼び出します。

### C. オブジェクトの登録
セッションを生成したら、Bookクラスのオブジェクトをデータベースに登録します。
オブジェクトを登録するにはBookクラスのインスタンスを生成し、__session.add()__メソッドの第１引数に渡します。
Integer型のカラムにprimary_key=Trueが指定されていると、そのカラムは自動的に番号が付けられます。

In [7]:
book = Book(title= 'やさしいPython', price='2580', memo = 'SBクリエイティブ')
session.add(book)
session.commit()

2020-03-24 14:19:31,745 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-24 14:19:31,747 INFO sqlalchemy.engine.base.Engine INSERT INTO books (title, price, memo) VALUES (?, ?, ?)
2020-03-24 14:19:31,748 INFO sqlalchemy.engine.base.Engine ('やさしいPython', '2580', 'SBクリエイティブ')
2020-03-24 14:19:31,749 INFO sqlalchemy.engine.base.Engine COMMIT


__add_all()__メソッドを使用すると一度に複数のBookインスタンスを渡してデータベースに登録出来ます。

In [8]:
session.add_all([
    Book(title= 'スッキリわかるJava入門', price='2680', memo = 'インプレス'),
    Book(title= '独習PHP', price='3520', memo = '翔泳社'),
])
session.commit()

2020-03-24 14:19:34,555 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-24 14:19:34,557 INFO sqlalchemy.engine.base.Engine INSERT INTO books (title, price, memo) VALUES (?, ?, ?)
2020-03-24 14:19:34,558 INFO sqlalchemy.engine.base.Engine ('スッキリわかるJava入門', '2680', 'インプレス')
2020-03-24 14:19:34,560 INFO sqlalchemy.engine.base.Engine INSERT INTO books (title, price, memo) VALUES (?, ?, ?)
2020-03-24 14:19:34,561 INFO sqlalchemy.engine.base.Engine ('独習PHP', '3520', '翔泳社')
2020-03-24 14:19:34,562 INFO sqlalchemy.engine.base.Engine COMMIT


### R. オブジェクトの取得
オブジェクトの登録ができたら、取得を行います。オブジェクトの取得はsession.query()を実行してQueryオブジェクトを生成します。
QueryオブジェクトはSQLを実行するためのオブジェクトです。

In [9]:
query = session.query(Book)

__all( )__メソッドを使うと全てのオブジェクトを取得できます。

In [10]:
for book in query.all():
    print (book.title)
    

2020-03-24 14:19:42,003 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-24 14:19:42,005 INFO sqlalchemy.engine.base.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books
2020-03-24 14:19:42,007 INFO sqlalchemy.engine.base.Engine ()
やさしいPython
スッキリわかるJava入門
独習PHP


__get( )__メソッドを使うとプライマリーキーを用いてオブジェクトを取得できます。

In [11]:
book = query.get(1)
print( book.title)

2020-03-24 14:19:47,164 INFO sqlalchemy.engine.base.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.id = ?
2020-03-24 14:19:47,164 INFO sqlalchemy.engine.base.Engine (1,)
やさしいPython


__filter( )__メソッドを使うと、条件を付けてオブジェクトを取得できます。

In [12]:
book = query.filter(Book.title == 'スッキリわかるJava入門').first()
print(book.title)

2020-03-24 14:19:50,473 INFO sqlalchemy.engine.base.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.title = ?
 LIMIT ? OFFSET ?
2020-03-24 14:19:50,474 INFO sqlalchemy.engine.base.Engine ('スッキリわかるJava入門', 1, 0)
スッキリわかるJava入門


つぎのように__filter( )__メソッドをつなげて、複数の条件を指定することができます。

In [13]:
book = query.filter(Book.title == '独習PHP').filter(Book.id == 3).first()
print(book.title)

2020-03-24 14:19:53,828 INFO sqlalchemy.engine.base.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.title = ? AND books.id = ?
 LIMIT ? OFFSET ?
2020-03-24 14:19:53,829 INFO sqlalchemy.engine.base.Engine ('独習PHP', 3, 1, 0)
独習PHP


__count( )__メソッドを使用すると、テーブル上のレコード数を取得できます。

In [14]:
count = query.count()
print(count)

2020-03-24 14:19:56,921 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books) AS anon_1
2020-03-24 14:19:56,922 INFO sqlalchemy.engine.base.Engine ()
3


### U. オブジェクトの更新
オブジェクトの更新は__.price=2500__のようにオブジェクトの値を変更して、__session.commit( )__を呼び出します。

In [15]:
book = query.get(1)
book.price = 2500
session.commit()
book = query.get(1)
print(book.price)

2020-03-24 14:20:00,353 INFO sqlalchemy.engine.base.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.id = ?
2020-03-24 14:20:00,354 INFO sqlalchemy.engine.base.Engine (1,)
2020-03-24 14:20:00,356 INFO sqlalchemy.engine.base.Engine UPDATE books SET price=? WHERE books.id = ?
2020-03-24 14:20:00,357 INFO sqlalchemy.engine.base.Engine (2500, 1)
2020-03-24 14:20:00,358 INFO sqlalchemy.engine.base.Engine COMMIT
2020-03-24 14:20:00,359 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-24 14:20:00,360 INFO sqlalchemy.engine.base.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.id = ?
2020-03-24 14:20:00,360 INFO sqlalchemy.engine.base.Engine (1,)
2500


### D. オブジェクトの削除
オブジェクトの削除は__session.delete( )__にBookインスタンスを渡してから、__session.commit( )__を呼び出します。

In [16]:
book = query.get(1)
session.delete(book)
session.commit()
book = query.get(1)
print(book)

for book in query.all():
    print(book.title)

2020-03-24 14:20:05,813 INFO sqlalchemy.engine.base.Engine DELETE FROM books WHERE books.id = ?
2020-03-24 14:20:05,813 INFO sqlalchemy.engine.base.Engine (1,)
2020-03-24 14:20:05,822 INFO sqlalchemy.engine.base.Engine COMMIT
2020-03-24 14:20:05,823 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-24 14:20:05,823 INFO sqlalchemy.engine.base.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.id = ?
2020-03-24 14:20:05,824 INFO sqlalchemy.engine.base.Engine (1,)
None
2020-03-24 14:20:05,826 INFO sqlalchemy.engine.base.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books
2020-03-24 14:20:05,826 INFO sqlalchemy.engine.base.Engine ()
スッキリわかるJava入門
独習PHP


In [17]:
book

<Book('独習PHP', '3520', '翔泳社')>